
#    311 Service Requests Data Analysis and Visualization
##   DATA 601 Project Report - Team 4 
###  Team Members : Anitha Joseph, Jincy Thomas, Megha Radhakrishnan Sanitha 


#### INTRODUCTION

    311 is a non-emergency government service that helps citizens to report issues and access city information. It can be accessed by phone, online, or through mobile apps and often includes multilingual support and services for vulnerable populations like the senior residents. The 311 system collects all requests, which are then routed to the responsible city department for resolution. It operates 24/7 and allows users to track the progress of their requests, ensuring transparency and accountability. Thus the 311 services make the lives of the residents easier and saves their time by eliminating the need to contact multiple departments for resolving their concerns. The user-friendly interface of 311 also makes the communication with the local government effortless, convenient and accessible for everyone.

    Analysis of 311 data over a long period of time helps us in determining the most frequent service requests which can be addressed more efficiently by allocating adequate resources. This can also be used to identify the efficiencies of different departments and the gaps where improvement is required. Seasonal patterns and long term trends can also be identified which can be used to predict the future demands and optimize the public services. By tracking types of requests, response times, and areas with frequent complaints, resources can be allocated more effectively, and operational inefficiencies can be identified and resolved. The residents will get a clear picture regarding the handling of their concerns which promotes transparency. Proactive planning is made possible, allowing for better resource distribution and improved service delivery. Ultimately, systemic issues can be identified, and solutions can be developed,leading to improved city planning and enhanced community satisfaction.


#### GUIDING QUESTIONS

    The major guiding questions for the analysis of 311 service requests and the the ways in which the insights from answering these questions can be utilized are as follows:
    1. Geographic Analysis
    ● Which community or location has the largest number of service requests?
    ● Are there any specific needs for certain areas?
    High need areas can be prioritized and resources can be allocated where they are most required.
    
    2. Seasonal Trends
    ● During which seasons do service requests occur most often?
    ● How do service requests change over seasons? Are there any identifiable patterns?
    Seasonal patterns are identified thus helping to focus resources effectively.
    
    3. Request Sources
    ● What is the primary source of service requests: phone calls or online submissions (web)?
    Interfaces can be upgraded based on user preferences.
    
    4. Types of Service Requests
    ● What is the service requested most frequently?
    Helps to identify and allocate adequate resources for the most needed service.
    
    5. Response Efficiency
    ● Which agency handles the most and least number of service requests?
    ● What is the average response rate and time for resolving for service requests?
    ● Who are the most efficient agents in terms of response and resolution times?
    ● How does the response efficiency vary across different years?
    Track the department efficiencies and identify areas where improvement is required.
    
    6. Trends Over Time
    ● How has the volume and type of service requests changed over the past five years?
    ● Are there noticeable trends in requests that could be used for future planning?
    Future demands can be anticipated, allowing for better preparedness.


#### DATASET

##### Data description

    The data for this analysis is sourced from The City of Calgary’s open data portal, specifically the "311 Service Requests -Services and Amenities" dataset (The City of Calgary,2025) . We have included public service requests submitted via 311 for 2 years- 2023 and 2024, consisting of 1062842 rows and 15 columns, structured in a tabular format. Each row represents an individual service request.
    
##### Format and Structure
    The data consists of several datasets depending on the city or organization's open data portal. Each dataset typically contains the following columns: 

    Column                Datatype   Description
    service_request_id     object    The unique identifier for an individual request.
    requested_date         object    The date the request was submitted.
    updated_date           object    The most recent date the request was updated.
    closed_date            object    The date the request was closed.
    status_description     object    The current status of the request (e.g. open, closed).
    source                 object    The channel used to submit the request
    service_name           object    The type of service requested.
    agency_responsible     object    The department responsible for this request.
    address               float64    The location of the service request (if applicable).
    comm_code              object    The community code associated with the service request location.
    comm_name              object    The community name associated with the service request location.
    location_type          object    The type of location information provided for this service request.
    longitude             float64    The longitude of the service request.
    latitude              float64    The latitude of the service request.
    point                  object    The spatial coordinates based on latitude and longitude.

    The data is sourced from The City of Calgary’s open data portal, 311 Service Requests - Services and Amenities.The data is publicly available and used with permission as per the open data policy. This data is provided by the City of Calgary at https://data.calgary.ca/Services-and-Amenities/311-Service-Requests/iahh-g8bj/about_data and all usagecomplies with their data usage and attribution requirements listed in the license URL https://data.calgary.ca/d/Open-Data-Terms/u45n-7awa .